### Vediamo come costruire e allenare un regressore lineare con PyTorch. Carichiamo il dataset Boston con scikit-learn

In [28]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.


In [29]:
from sklearn.datasets import fetch_openml
boston = fetch_openml(name="boston", version=1)
print(boston.DESCR)

**Author**:   
**Source**: Unknown - Date unknown  
**Please cite**:   

The Boston house-price data of Harrison, D. and Rubinfeld, D.L. 'Hedonic
prices and the demand for clean air', J. Environ. Economics & Management,
vol.5, 81-102, 1978.   Used in Belsley, Kuh & Welsch, 'Regression diagnostics
...', Wiley, 1980.   N.B. Various transformations are used in the table on
pages 244-261 of the latter.
Variables in order:
CRIM     per capita crime rate by town
ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
INDUS    proportion of non-retail business acres per town
CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
NOX      nitric oxides concentration (parts per 10 million)
RM       average number of rooms per dwelling
AGE      proportion of owner-occupied units built prior to 1940
DIS      weighted distances to five Boston employment centres
RAD      index of accessibility to radial highways
TAX      full-value property-tax rate per $10

c:\Users\simon\anaconda3\Lib\site-packages\sklearn\datasets\_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Il dataset consiste in 506 osservazioni, ognuna di esse ha 13 valori che costituiscono rilevamenti di vicinati di Boston.


In [30]:
X = boston.data.to_numpy().astype(float)
Y = boston.target.to_numpy().astype(float)

X sono i rilevamenti, quindi sono 506 quartieri con ognuno contenente 13 rilevamenti
Y sono i valori mediani delle case in ognuno dei vicinati espressi in multipli di 1000

Calcoliamo una permutazione casuale degli indici.

In [31]:
import numpy as np
import torch

np.random.seed(123)
torch.random.manual_seed(123)

idx = np.random.permutation(len(X))

# Applichiamo la stessa permutazione a rilevamenti e target
X = X[idx]
Y = Y[idx]

#### Adesso suddividiamo il dataset in training e testing selezionando i primi 50 valori per formare il testing set.

#### Trasformiamo gli array in tensori e inseriamoli in delle variabili.

In [32]:
X_training = torch.Tensor(X[50:])
Y_training = torch.Tensor(Y[50:])
X_testing = torch.Tensor(X[:50])
Y_testing = torch.Tensor(Y[:50])

### Q1: Perchè abbiamo effetuato una permutazione casuale dei dati prima di dividerli in training e test?


Perchè così se il dataset ha un bias andiamo a renderlo omogeneo e dividiamo correttamente tra train e test

Il modello di regressione dipende da 14 parametri:
- 13 sono le features in ingresso
- 1 l'intercetta della retta
Iniializziamo l'array dei pesi $\theta$ casualmente con distribuzione normale media zero varianza 0.1 così che inizialmente abbia valori sia positivi che negativi 

Definiamo una variabile per l'intercetta, ovvero theta_0

Il bias in un modello di regressione lineare è l'intercetta della retta, ovvero il valore che la funzione di regressione assume quando tutte le variabili indipendenti sono pari a zero.

La funzione del modello è:
$$
y(x) = \theta_1x_1+...+\theta_{13}x_{13}+\theta_0
$$

In [33]:
theta = torch.Tensor(13) # Tensore da 13 unità
theta_0 = torch.Tensor(1) # Bias

theta.requires_grad_(True)
theta_0.requires_grad_(True)

theta.data.normal_(0,0.1)
theta_0.data.normal_(0,0.1)

print(theta)
print(theta_0)

tensor([-0.0111,  0.0120, -0.0370, -0.0240, -0.1197,  0.0209, -0.0972, -0.0755,
         0.0324, -0.0109,  0.0210, -0.0391,  0.0235], requires_grad=True)
tensor([0.0665], requires_grad=True)


In PyTorch, l'attributo requires_grad è fondamentale per il calcolo automatico del gradiente e l'ottimizzazione dei modelli di machine learning.

Quando un tensore ha requires_grad=True, PyTorch tiene traccia delle operazioni fatte su di esso, costruendo un grafo computazionale dinamico. Questo permette di calcolare automaticamente i gradienti tramite backpropagation, essenziali per l'ottimizzazione del modello

In [34]:
def linear_regression(input, theta, theta_0):
    return input.mul(theta).sum(1)+theta_0

In [35]:
res = X_training.mul(theta)
print(res, res.shape)

tensor([[-6.2912e-04,  4.8145e-01, -2.3694e-01,  ...,  3.7018e-01,
         -1.5513e+01,  8.2946e-02],
        [-6.7404e-04,  0.0000e+00, -9.0930e-02,  ...,  3.7439e-01,
         -1.5130e+01,  3.0899e-01],
        [-7.0198e-03,  0.0000e+00, -3.0088e-01,  ...,  4.4170e-01,
         -1.5513e+01,  1.9409e-01],
        ...,
        [-3.9141e-03,  0.0000e+00, -2.7279e-01,  ...,  4.1225e-01,
         -1.5513e+01,  1.8093e-01],
        [-1.0241e-01,  0.0000e+00, -6.6904e-01,  ...,  4.2487e-01,
         -1.5513e+01,  5.5454e-01],
        [-5.0783e-02,  0.0000e+00, -6.6904e-01,  ...,  4.2487e-01,
         -1.3863e+01,  1.6730e-01]], grad_fn=<MulBackward0>) torch.Size([456, 13])


Abbiamo moltiplicato ogni riga di X_training per il vettore theta, quindi stiamo formando l'equazione di regressione lineare. Infine, sommiamo lungo l'asse.

Questo restituisce per ogni campione la combinazione lineare, adesso manca solo theta_0

In [38]:
res = X_training.mul(theta).sum(1)+theta_0

In [39]:
y = linear_regression(X_training, theta, theta_0)
print(y[:10])

tensor([-20.8190, -23.3360, -24.6247, -31.4600, -27.4998, -28.1950, -27.1047,
        -26.2724, -18.8539, -31.4594], grad_fn=<SliceBackward0>)


Confrontiamo questi valori con i veri valori quindi Ground Truth

In [40]:
print(Y_training[:10])

tensor([32.4000, 29.6000, 20.4000, 12.3000, 19.1000, 14.9000, 17.8000,  8.8000,
        35.4000, 11.5000])


I valori sono molto diversi ed è normale perchè il modello per ora usa pesi inizializzati a caso, adesso dobbiamo ottimizzare il modello tramite la discesa del gradiente. Per farlo definiamo una funzione di loss cioè una funzione differenziabile che esprime l'errore che il modello commette.

Qui useremo Mean Squared Error.
$$
MSE(y, \hat y) = \frac 1N \sum_i^N(\hat y_i - y_i)^2
$$

dove $\hat y$ sono le predizioni, mentre $y$ sono le ground truth e N il numero di campioni

In [41]:
def loss(input, target):
    return ((input-target)**2).mean()

In [42]:
print(loss(y,Y_training))

tensor(2274.0801, grad_fn=<MeanBackward0>)


è un numero molto grande perchè le predizioni sono molto distanti dalle GT